# PRMT-2608 Pre GP2GP failures from MI data for 2 CCGs

## Context
We have been in contact and working with a ccg who have been looking at issues that affect GP2GP, including pre-GP2GP issues. 

They have expressed a need to be able to see data about the pre-GP2GP part of the process to help them in their work to identify issues and the size of issues for their specific practices within the CCGs. 

This story is to create a one-off report for 01T and 01V CCGs (this is from the MI data source and not the spine logs). 


## Notes

Data downloaded from Splunk using the following query (Feb 2022):
```
index="gp2gp_nms_prod" sourcetype="gp2gpmi-rr"
| table *
```

In [1]:
import pandas as pd
import numpy as np
import paths, data
from data.practice_metadata import read_asid_metadata

In [2]:
def convert_to_float(val):
    try:
        return int(val)
    except:
        return val

mi_data_file_location = "s3://prm-gp2gp-notebook-data-prod/PRMT-2608-mi-report-for-pre-gp2gp-feb-2022/MI_Feb_2022.csv"

dates_fields = ["RegistrationTime", "RequestFailureTime", "RequestTime", "ExtractTime", "ExtractAckTime", "ExtractAckFailureTime"]
practice_registrations = pd.read_csv(mi_data_file_location, parse_dates=dates_fields).fillna("None")

practice_registrations["RequestErrorCode"] = practice_registrations["RequestErrorCode"].apply(convert_to_float)
practice_registrations["RequestFailureType"] = practice_registrations["RequestFailureType"].apply(convert_to_float)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,17,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create a unique key and drop duplicates
practice_registrations["UniqueKey"] = (practice_registrations["RegistrationSmartcardUID"].astype(str)
                                       + "-"
                                       + practice_registrations["RegistrationTime"].astype(str))

practice_registrations = (
    practice_registrations
        .sort_values(by="_time", ascending=True)
        .drop_duplicates(subset=["UniqueKey"], keep="last")
    )

In [4]:
# Check total number of registrations
practice_registrations.shape[0]

427741

In [5]:
# Breakdown of all registrations that did not trigger GP2GP
def has_conversation_id(value):
    if value=="None":
        return 0
    else:
        return 1
    
practice_registrations["Error scenario"] = practice_registrations[["RequestFailurePoint", "RequestFailureType", "RequestErrorCode"]].apply(lambda x: '_'.join(x.astype(str)), axis=1)
practice_registrations["Triggered GP2GP"] = practice_registrations.apply(lambda row: has_conversation_id(row["ConversationID"]), axis=1)

## CCG 1 - 01T

In [6]:
ccg_1_json = pd.read_json("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?TargetOrgId=01T&RelTypeId=RE4&RelStatus=active&Limit=1000")
list_of_practice_ods_codes_for_ccg_1 = ccg_1_json["Organisations"].apply(lambda practice: practice.get("OrgId")).tolist()

In [7]:
transfers_for_practices_within_ccg_1 = practice_registrations["RequestorODS"].apply(lambda ods: ods in list_of_practice_ods_codes_for_ccg_1)
transfers_for_practices_within_ccg_1 = practice_registrations[transfers_for_practices_within_ccg_1]
transfers_for_practices_within_ccg_1.shape

(672, 54)

In [8]:
did_not_trigger_gp2gp_bool_ccg_1 = transfers_for_practices_within_ccg_1["Triggered GP2GP"]==False
practice_registrations_no_gp2gp_ccg_1 = transfers_for_practices_within_ccg_1[did_not_trigger_gp2gp_bool_ccg_1]

registrations_that_didnt_trigger_gp2gp_grouped_by_failures_ccg_1 = (
    practice_registrations_no_gp2gp_ccg_1
        .groupby(by=["RequestFailurePoint", "RequestFailureType", "RequestErrorCode", "Triggered GP2GP"])
        .agg({"UniqueKey": "count"})
        .rename(columns={"UniqueKey": "count"})
        .sort_values(by="count", ascending=False)
    )
registrations_that_didnt_trigger_gp2gp_grouped_by_failures_ccg_1

,,,,count
RequestFailurePoint,RequestFailureType,RequestErrorCode,Triggered GP2GP,
60,5,None,0,178
0,0,None,0,67
60,2,None,0,17
40,4,24,0,4
0,None,None,0,1
60,0,20,0,1


In [9]:
# Breakdown of all registrations that did not trigger GP2GP by practice and error scenario
practice_registrations_with_pre_gp2gp_error_scenarios_ccg_1 = practice_registrations_no_gp2gp_ccg_1.pivot_table(index=["RequestorODS"], 
        columns=["Error scenario"], 
        values="UniqueKey", 
        aggfunc="count").fillna(0).astype(int)

In [10]:
# Create table with total number of registrations, number of registrations that triggered GP2GP and pre-GP2GP error scenario counts by practice
practice_registrations_summary_ccg_1 = transfers_for_practices_within_ccg_1.groupby("RequestorODS").agg({"UniqueKey":"count", "Triggered GP2GP": "sum"}).rename(columns={"UniqueKey": "Total registrations"})

all_practice_registrations_with_pre_gp2gp_breakdown_ccg_1 = practice_registrations_summary_ccg_1.join(practice_registrations_with_pre_gp2gp_error_scenarios_ccg_1, how="left").fillna(0).astype(int)

In [11]:
# Add practice names (via ASID lookup) to the table above
asid_lookup = read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v3/2022/3/organisationMetadata.json")[["practice_ods_code", "practice_name"]]
asid_lookup = asid_lookup.set_index("practice_ods_code")

practice_registrations_with_pivot_with_practice_names_ccg_1 = all_practice_registrations_with_pre_gp2gp_breakdown_ccg_1.join(asid_lookup, on="RequestorODS", how="left").fillna("None")
practice_registrations_with_pivot_with_practice_names_ccg_1 = practice_registrations_with_pivot_with_practice_names_ccg_1.drop_duplicates()
practice_registrations_with_pivot_with_practice_names_ccg_1 = practice_registrations_with_pivot_with_practice_names_ccg_1.reset_index().rename(columns={"practice_name": "Requesting practice name", "RequestorODS": "Requesting practice ODS"})

column_order_ccg_1 = ["Requesting practice name", "Requesting practice ODS", "Total registrations", "Triggered GP2GP", "60_5_None", "60_2_None", "60_0_20", "40_4_24", "0_0_None", "0_None_None"]
practice_registrations_with_pivot_with_practice_names_ccg_1 = practice_registrations_with_pivot_with_practice_names_ccg_1[column_order_ccg_1].sort_values(by="Requesting practice name", ascending=True)

In [13]:
with pd.ExcelWriter("PRMT-2608-Pre-GP2GP-failures-from-MI-data-Feb-2022-CCG-01T.xlsx") as writer:
     practice_registrations_with_pivot_with_practice_names_ccg_1.to_excel(writer, sheet_name="Pre-GP2GP errors 01T",index=False)

# CCG 2 - 01V

In [20]:
ccg_2_json = pd.read_json("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?TargetOrgId=01V&RelTypeId=RE4&RelStatus=active&Limit=1000")
list_of_practice_ods_codes_for_ccg_2 = ccg_2_json["Organisations"].apply(lambda practice: practice.get("OrgId")).tolist()

In [21]:
transfers_for_practices_within_ccg_2 = practice_registrations["RequestorODS"].apply(lambda ods: ods in list_of_practice_ods_codes_for_ccg_2)
transfers_for_practices_within_ccg_2 = practice_registrations[transfers_for_practices_within_ccg_2]
transfers_for_practices_within_ccg_2.shape

(815, 54)

In [22]:
did_not_trigger_gp2gp_bool_ccg_2 = transfers_for_practices_within_ccg_2["Triggered GP2GP"]==False
practice_registrations_no_gp2gp_ccg_2 = transfers_for_practices_within_ccg_2[did_not_trigger_gp2gp_bool_ccg_2]

registrations_that_didnt_trigger_gp2gp_grouped_by_failures_ccg_2 = (
    practice_registrations_no_gp2gp_ccg_2
        .groupby(by=["RequestFailurePoint", "RequestFailureType", "RequestErrorCode", "Triggered GP2GP"])
        .agg({"UniqueKey": "count"})
        .rename(columns={"UniqueKey": "count"})
        .sort_values(by="count", ascending=False)
    )
registrations_that_didnt_trigger_gp2gp_grouped_by_failures_ccg_2

,,,,count
RequestFailurePoint,RequestFailureType,RequestErrorCode,Triggered GP2GP,
60,5,None,0,143
0,0,None,0,62
40,4,24,0,13
60,2,None,0,13
40,3,24,0,3
20,0,None,0,2


In [23]:
# Breakdown of all registrations that did not trigger GP2GP by practice and error scenario
practice_registrations_with_pre_gp2gp_error_scenarios_ccg_2 = practice_registrations_no_gp2gp_ccg_2.pivot_table(index=["RequestorODS"], 
        columns=["Error scenario"], 
        values="UniqueKey", 
        aggfunc="count").fillna(0).astype(int)

In [24]:
# Create table with total number of registrations, number of registrations that triggered GP2GP and pre-GP2GP error scenario counts by practice
practice_registrations_summary_ccg_2 = transfers_for_practices_within_ccg_2.groupby("RequestorODS").agg({"UniqueKey":"count", "Triggered GP2GP": "sum"}).rename(columns={"UniqueKey": "Total registrations"})

all_practice_registrations_with_pre_gp2gp_breakdown_ccg_2 = practice_registrations_summary_ccg_2.join(practice_registrations_with_pre_gp2gp_error_scenarios_ccg_2, how="left").fillna(0).astype(int)

In [26]:
# Add practice names (via ASID lookup) to the table above
# asid_lookup = read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v3/2022/3/organisationMetadata.json")[["practice_ods_code", "practice_name"]]
# asid_lookup = asid_lookup.set_index("practice_ods_code")
# already done above

practice_registrations_with_pivot_with_practice_names_ccg_2 = all_practice_registrations_with_pre_gp2gp_breakdown_ccg_2.join(asid_lookup, on="RequestorODS", how="left").fillna("None")
practice_registrations_with_pivot_with_practice_names_ccg_2 = practice_registrations_with_pivot_with_practice_names_ccg_2.drop_duplicates()
practice_registrations_with_pivot_with_practice_names_ccg_2 = practice_registrations_with_pivot_with_practice_names_ccg_2.reset_index().rename(columns={"practice_name": "Requesting practice name", "RequestorODS": "Requesting practice ODS"})

column_order_ccg_2 = ["Requesting practice name", "Requesting practice ODS", "Total registrations", "Triggered GP2GP", "60_5_None", "60_2_None", "40_4_24", "40_3_24", "20_0_None", "0_0_None"]
practice_registrations_with_pivot_with_practice_names_ccg_2 = practice_registrations_with_pivot_with_practice_names_ccg_2[column_order_ccg_2].sort_values(by="Requesting practice name", ascending=True)

In [27]:
with pd.ExcelWriter("PRMT-2608-Pre-GP2GP-failures-from-MI-data-Feb-2022-CCG-01V.xlsx") as writer:
     practice_registrations_with_pivot_with_practice_names_ccg_2.to_excel(writer, sheet_name="Pre-GP2GP errors 01V",index=False)

---------------------------

**RequestFailurePoint:**
- 0 = No failure
- 10 = PDS trace
- 20 = PDS update

- 30 = SDS lookup Practice (not used)
- 40 = SDS lookup ASID
- 50 = SDS lookup Contract Props
- 60 = Send Request
- 70 = Manual Request

**RequestFailureType:**
- 0 = Attempted
- 1 = Sent
- 2 = Not Sent - Patient at current practice
- 3 = Not Sent - Patient known at current practice transferring from non-GP2GP practice
- 4 = Not Sent - Patient not known at current practice transferring from a non-GP2GP practice
- 5 = Not Sent – Patient has no previous practice registered
- 6 = Negative acknowledgement received

**RequestErrorCode:**
- 3 = Record available but cannot be sent - DEPRECATED
- 8 = The system’s configuration prevents it from processing this message - DEPRECATED
- 20 = Spine system responded with an error
- 24 = SDS lookup provided zero or more than one result to the query for each interaction
- 25 = Large messages rejected due to timeout duration reached of overall transfer